# Block 7 Übung 1: Orientierung in den Daten der MIMIC-Tabelle Chartevents

## Bibliotheken & Konfiguration

In [1]:
packages <- c("readr", "dplyr")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-Datenanalyse-Visualisierung/master/Rohdaten/"

Lade nötiges Paket: readr

Lade nötiges Paket: dplyr


Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union




[[1]]
[1] TRUE

[[2]]
[1] TRUE

## MIMIC III-Basisdaten laden

In [2]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Chartevents laden

In [3]:
# Chartevents laden
mimic.chartevents.raw <- read_csv(paste(base_url, "mimic-iii-demo/CHARTEVENTS.csv", sep=""),
                                  col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), itemid = col_integer(), charttime = col_datetime(format = ""), storetime = col_datetime(format = ""), cgid = col_double(), value = col_character(), valuenum = col_double(), valueuom = col_character(), warning = col_double(), error = col_double(), resultstatus = col_character(), stopped = col_character()))
head(mimic.chartevents.raw)

row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
<int>,<int>,<int>,<int>,<int>,<dttm>,<dttm>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
5279021,40124,126179,279554,223761,2130-02-04 04:00:00,2130-02-04 04:35:00,19085,95.9,95.9,?F,0,0,NA,NA
5279022,40124,126179,279554,224695,2130-02-04 04:25:00,2130-02-04 05:55:00,18999,2222221.7,2222221.7,cmH2O,0,0,NA,NA
5279023,40124,126179,279554,220210,2130-02-04 04:30:00,2130-02-04 04:43:00,21452,15,15.0,insp/min,0,0,NA,NA
5279024,40124,126179,279554,220045,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,94,94.0,bpm,0,0,NA,NA
5279025,40124,126179,279554,220179,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,163,163.0,mmHg,0,0,NA,NA
5279026,40124,126179,279554,220180,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,81,81.0,mmHg,0,0,NA,NA


### Bezeichner ergänzen

In [4]:
# Bezeichner laden
mimic.d_items.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ITEMS.csv", sep=""),
                              col_types = cols(row_id = col_integer(), itemid = col_integer(), label = col_character(), abbreviation = col_character(), dbsource = col_character(), linksto = col_character(), category = col_character(), unitname = col_character(), param_type = col_character(), conceptid = col_character()))

# Bezeichner hinzufügen
mimic.chartevents.annotated <- mimic.chartevents.raw %>%
  inner_join(mimic.d_items.raw %>% select(itemid, category, label), by = "itemid")

head(mimic.chartevents.annotated)

row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,category,label
<int>,<int>,<int>,<int>,<int>,<dttm>,<dttm>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
5279021,40124,126179,279554,223761,2130-02-04 04:00:00,2130-02-04 04:35:00,19085,95.9,95.9,?F,0,0,NA,NA,Routine Vital Signs,Temperature Fahrenheit
5279022,40124,126179,279554,224695,2130-02-04 04:25:00,2130-02-04 05:55:00,18999,2222221.7,2222221.7,cmH2O,0,0,NA,NA,Respiratory,Peak Insp. Pressure
5279023,40124,126179,279554,220210,2130-02-04 04:30:00,2130-02-04 04:43:00,21452,15,15.0,insp/min,0,0,NA,NA,Respiratory,Respiratory Rate
5279024,40124,126179,279554,220045,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,94,94.0,bpm,0,0,NA,NA,Routine Vital Signs,Heart Rate
5279025,40124,126179,279554,220179,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,163,163.0,mmHg,0,0,NA,NA,Routine Vital Signs,Non Invasive Blood Pressure systolic
5279026,40124,126179,279554,220180,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,81,81.0,mmHg,0,0,NA,NA,Routine Vital Signs,Non Invasive Blood Pressure diastolic


## Aufgabe: Finden Sie häufig verwendete Items für den Blutdruck und Puls!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) anwenden.
* zugehöriges Notebook: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb)

Filtern Sie die Rohdaten mit der `filter()`-Funktion nach Inhalten, die Sie für Blutdruck & Puls erwarten würden (z.B. Maßeinheiten in der Spalte `valueuom` oder Bezeichner in der Spalte `label`. Finden Sie häufig verwendete Attribute, indem Sie die Anzahl der Zeilen mit den `group_by()`- und `summarize()`-Funktionen aggregieren.

Tipps zur `filter()`-Funktion:
* Wenn Sie mehrere mit Komma getrennte Kriterien angeben, werden sie mit logisch "und" verknüpft
* Sie können Kriterien mit dem `|`-Operator auch als logisch "oder" kombinieren
* Sie können mit dem `%in%`-Operator auch nach mehreren verschiedenen Ausprägungen eines Textes suchen (z.B. `filter(farbe %in% c("rot", "blau"))`)
* Sie können auch nach Teilstrings suchen, indem Sie die Funktion `grepl("Suchbegriff", Spalte)` nutzen

Sonstige Tipps:
* Denken Sie daran, dass Sie bei der `summarize()`-Funktion die Option `.groups = "keep"` angeben müssen, wenn die Gruppierungsmerkmale beibehalten werden sollen. Sie erhalten sonst nur das aggregierte Ergebnis als Zahl, aber nicht die (meistens benötigten) Merkmale, nach denen zuvor gruppiert wurde.

In [5]:
# Mögliche Ausprägungen für den Blutdruck finden
mimic.chartevents.annotated %>% filter(valueuom == "mmHg", grepl("systolic", tolower(label)) | grepl("diastolic", tolower(label))) %>%
    group_by(itemid, label, valueuom) %>%
    summarize(n = n(), .groups = "keep")

itemid,label,valueuom,n
<int>,<chr>,<chr>,<int>
51,Arterial BP [Systolic],mmHg,4506
442,Manual BP [Systolic],mmHg,1
455,NBP [Systolic],mmHg,3208
492,PAP [Systolic],mmHg,1446
6701,Arterial BP #2 [Systolic],mmHg,27
8368,Arterial BP [Diastolic],mmHg,4504
8440,Manual BP [Diastolic],mmHg,1
8441,NBP [Diastolic],mmHg,3205
8448,PAP [Diastolic],mmHg,1445


In [6]:
# Mögliche Ausprägungen für den Puls finden
mimic.chartevents.annotated %>% filter(valueuom == "bpm") %>%
    group_by(itemid, label, valueuom) %>%
    summarize(n = n(), .groups = "keep")

itemid,label,valueuom,n
<int>,<chr>,<chr>,<int>
63,BIPAP - BPM,bpm,1
220045,Heart Rate,bpm,8094
220046,Heart rate Alarm - High,bpm,883
220047,Heart Rate Alarm - Low,bpm,885
224422,Spont RR,bpm,65
224751,Temporary Pacemaker Rate,bpm,2
226764,HrApacheIIValue,bpm,1
